<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo11n_Shikibo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00


In [6]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

sending incremental file list
dataset.zip
          1.19G 100%   85.03MB/s    0:00:13 (xfr#1, to-chk=0/1)
Archive:  /content/dataset.zip
   creating: /content/dataset/
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/images/
  inflating: /content/dataset/images/CAM1_250801135246.jpg  
  inflating: /content/dataset/images/CAM1_250804080357.jpg  
  inflating: /content/dataset/images/CAM1_250804114157.jpg  
  inflating: /content/dataset/images/CAM1_250805024725.jpg  
  inflating: /content/dataset/images/CAM1_250805050017.jpg  
  inflating: /content/dataset/images/CAM1_250805133915.jpg  
  inflating: /content/dataset/images/CAM1_250806104530.jpg  
  inflating: /content/dataset/images/CAM1_250806104734.jpg  
  inflating: /content/dataset/images/CAM1_250806111847.jpg  
  inflating: /content/dataset/images/CAM1_250806131946.jpg  
  inflating: /content/dataset/images/CAM1_250807090505.jpg  
  inflating: /content/dataset/images/CAM1_250820133007.jpg  
  inflating: /conten

In [7]:
model_path = "/content/drive/MyDrive/yolo11n.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [8]:
import yaml

# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [9]:
import numpy as np
import os
from ultralytics import YOLO

def train_yolo(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLO11n model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLO11 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolo11_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=4,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
#model_path = "/content/drive/MyDrive/yolo12s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 300  # エポック数
batch_size = 32  # バッチサイズ (Reduced to mitigate OOM error)
patience=40
label_smoothing=0.0
img_size = 960 # 画像サイズ
iou_threshold = 0.5  # IoU 閾値
lr0=0.003
weight_decay=0.001
label_smoothing=0.05
mosaic = 0.0  # モザイクの確率
mixup = 0.0  # Mixup の確率
copy_paste=0.05
translate=0.05
close_mosaic=10
perspective=0.0
shear=0.
fliplr=0.1
flipud=0.0
rect=True
cos_lr=True
multi_scale=True
seed=0
output_dir = "runs_experiment"  # 出力ディレクトリ



# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLO の学習実行
train_yolo(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading model from /content/drive/MyDrive/yolo11n.pt
Training YOLO11 with 200 epochs...
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=4, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, f

In [10]:
# best.pt のパス
best_model_path = 'runs_experiment/yolo11_training/weights/best.pt'

# best.pt が存在すれば自動でダウンロード
if os.path.exists(best_model_path):
    files.download(best_model_path)
else:
    print("best.pt が見つかりませんでした。")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>